### Ordinary Differential Equations

The single pendulum problem can be considered as an ordinary differential equation of the form

\begin{align}
  \frac{d \vec{u}(t)}{dt} & = \vec{f}(t,\vec{u}(t))
\end{align}

where $\vec{u}(t) (= y(t))$ is the state vector at time $t$ consisting of position $x(t)$ and velocity $v(t)$. 

In general, ordinary differential equations cannot be solved analytically (sometimes they can).

$\leadsto$ We need numerical solution methods

# Simulation of the single pendulum


\begin{align*}
 \varphi(t):&  \quad \text{Displacement from the equilibrium position} && \quad [\varphi] = 1 rad\\
 v(t):&  \quad \text{Orbital velocity of the pendulum} && \quad [v] = 1 m/s \\
 \\
 g : & \quad \text{Gravitational constant} && \quad [g] = 1 m/s^2  \\
 l : & \quad \text{Length of the pendulum} && \quad [l] = 1 m  \\
 k =\frac{b}{m}: & \quad \text{Damping constant of the pendulum} && \quad [k] = 1 / s
\end{align*}

\begin{align*}
    \frac{d \varphi(t)}{dt} & = v(t) / l \\
    \frac{d v(t)}{dt} & = -g\sin(\varphi(t)) - k v(t)
\end{align*}

## Application of the explicit Euler method:

#### Formulation as general ordinary differential equation (ODE):

\begin{align}
  \frac{d \vec{u}(t)}{dt} & = \vec{f}(\vec{u}(t))
\end{align}
with
$$
\vec{u}(t) = 
\left( 
\begin{array}{c}
\varphi(t) \\ v(t)
\end{array}
\right)
$$
and
$$
\vec{f}(\vec{u}) = \vec{f}(\varphi,v) = \left( 
\begin{array}{c}
 v / l\\ 
 - g \sin(\varphi) - k v
\end{array}
\right)
= \left( 
\begin{array}{c}
 \vec{u}_2 / l  \\
 - g \sin(\vec{u}_1) - k \vec{u}_2
\end{array}
\right)
$$



This turns the general explicit Euler method
\begin{align}
  \vec{u}^{n+1} & = \vec{u}^{n} + \Delta t \cdot \vec{f}(\vec{u}^n)
\end{align}
into the following rule:
\begin{align}
  \varphi^{n+1} & = \varphi^{n} + \Delta t \cdot v^{n} / l \\
  v^{n+1} & = v^{n} + \Delta t \cdot ( - g \sin(\varphi^n) - k v^n  )
\end{align}

#### Initial values / materials parameters

* Initial valules:

In [ ]:
initialvals = { "phi" : 1.0, "v" : 0.0}

* Materials parameters:

In [ ]:
matparam =  { "g" : 1.0, "l" : 1.0, "k" : 0.0, "m" : 1.0}

### One-step Euler method:

Task:
 * Given the state $\varphi^n, v^n$ (i.e. $\vec{u}^n$), compute $\varphi^{n+1}, v^{n+1}$ (i.e. $\vec{u}^{n+1}$).

In [ ]:
from math import sin, cos, pi
def OneStep_ExplicitEuler(u, matparam, simparam):
    phi0, v0 = u
    u[0] += simparam["dt"] * v0 / matparam["l"] 
    u[1] -= simparam["dt"] * ( matparam["g"] * sin(phi0) + matparam["k"] * v0)

### The timestep method:

In [ ]:
from numpy import array
from plot import plot
%matplotlib notebook
def ODESolve(initialvals, matparam, simparam):
    timehistory = []
    poshistory = []
    velhistory = []
    
    u = array([initialvals["phi"],initialvals["v"]],dtype=float)
    t = simparam["t0"]

    timehistory.append(t)
    poshistory.append(u[0])
    velhistory.append(u[1])
    
    dt = simparam["dt"]
    cnt = 0
    while t < simparam["tend"]:
        simparam["method"](u,matparam,simparam)
        t = t + dt
        timehistory.append(t)
        poshistory.append(u[0])
        velhistory.append(u[1])
    plot(timehistory,poshistory,velhistory)
    return u

### Calculation of the energy

In [ ]:
def Energy(matparam,phi,v):
    return matparam["g"] * matparam["l"] * matparam["m"] * (1-cos(phi)) + 0.5 * matparam["m"] * v**2

### Simulation with the explicit Euler method

In [ ]:
#matparam =  { "g" : 1, "l" : 1, "k" : 0.0, "m" : 1.0}
simparam = { "dt" : 0.1, "t0" : 0, "tend" : 4*pi, "method" : OneStep_ExplicitEuler}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))

#### Task:
  * Simulate for different cases of the parameters $g,~l$ and $k$ for several oscillation periods.
  * Always reduce $\Delta t$ until the result no longer visibly changes.
  * What do you observe?

#### Task:
  * Also consider initial angles of $\pi$ or different initial velocities
  * Why are not two full oscillation periods completed after $4\pi$? 

#### Task: 
  * Consider $k = 0$. 
  * What total energy remains in the system after some simulated oscillations?
  * What total energy should theoretically remain in the system?
  * How does the error in energy depend on the number of oscillations and the time step $\Delta t$?  

### The Symplectic Euler Method

The explicit Euler method is often modified for multi-body problems to show better long-term behavior:

\begin{align}
  v^{n+1} & = v^{n} + \Delta t \cdot ( - g \sin(\varphi^{n}) - k v^n  ) \\
  \varphi^{n+1} & = \varphi^{n} + \Delta t \cdot v^{n+1} / l
\end{align}

The only difference to the explicit Euler method is that the updated $v$ is used for the update of $\varphi$.

In [ ]:
def OneStep_SymplecticEuler(u, matparam, simparam):
    phi0, v0 = u
    u[1] -= simparam["dt"] * ( matparam["g"] * sin(phi0) + matparam["k"] * v0)
    u[0] += simparam["dt"] * u[1]/matparam["l"]

#### Simulation with the symplectic Euler method

In [ ]:
#matparam =  { "g" : 1, "l" : 1, "k" : 0.0, "m" : 1.0}
simparam = { "dt" : 0.01, "t0" : 0, "tend" : 4*pi, "method" : OneStep_SymplecticEuler}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))

#### Task (as above): 
  * Consider $k = 0$. 
  * What total energy remains in the system after some simulated oscillations?
  * What total energy should theoretically remain in the system?
  * How does the error in energy depend on the number of oscillations and the time step $\Delta t$?  

#### Task: 
  * Implement the improved Euler method / Heun's method by creating a new function 
  `OneStep_Heun(...)` similar to `OneStep_ExplicitEuler(...)`. To do this, simply fill out the code block below and remove the line `raise Exception("Heun not yet implemented")`.
  * Compare Heun's method with the methods covered above. What do you observe (accuracy/cost/long-term behavior)?

In [ ]:
def OneStep_Heun(u, matparam, simparam):
    raise Exception("Heun not yet implemented")
    phi0, v0 = u

    uhalf0 = ...
    uhalf1 = ... 
    
    u[0] = ...
    u[1] = ...   
    

#### Simulation with Heun's method

In [ ]:
#matparam =  { "g" : 1, "l" : 1, "k" : 0.0, "m" : 1.0}
simparam = { "dt" : 0.1, "t0" : 0, "tend" : 4*pi, "method" : OneStep_Heun}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))

#### Additional Task: 
  * Implement another method
  `OneStep_StoermerVerlet(...)` similar to `OneStep_ExplicitEuler(...)`.  To do this, simply fill out the code block below and remove the line `raise Exception("Heun not yet implemented")`.
  The Störmer-Verlet method is defined as follows:
  \begin{align}
  \varphi^{n+\frac12} & = \varphi^{n} + \Delta t/2 ~ v^{n} / l \\
  v^{n+\frac12} & = v^{n} + \Delta t/2 ~  ( - g \sin(\varphi^{n+\frac12}) - k v^n  ) \\
  v^{n+1} & = v^{n+\frac12} + \Delta t/2 ~  ( - g \sin(\varphi^{n+\frac12}) - k v^{n+\frac12} ) \\
  \varphi^{n+1} & = \varphi^{n+\frac12} + \Delta t/2 ~ v^{n+1} / l \\
\end{align}
    
  * What advantages do you observe for this method?

In [ ]:
def OneStep_StoermerVerlet(u, matparam, simparam):
    raise Exception("Heun not yet implemented")
    phi0, v0 = u
    u[0] = ...
    u[1] = ...

#### Simulation with the Stoermer-Verlet method

In [ ]:
matparam =  { "g" : 1, "l" : 1, "k" : 0.0, "m" : 1.0}
simparam = { "dt" : 0.1, "t0" : 0, "tend" : 4*pi, "method" : OneStep_StoermerVerlet}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))